In [1]:
import pandas as pd
import os
import json
import numpy as np
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")

## Top 20 video mỗi tuần


In [2]:
# Define data types for some columns
dtypes = {
    "id": np.object_,
    "video.id": np.object_,
    "video.videoID": np.object_,
    "user.commerceUserInfo.category": np.object_,
    "user.commerceUserInfo.categoryButton": np.object_,
    "BAInfo": np.object_,
    "user.bioLink.link": np.object_,
}

In [3]:
top_week_df = pd.read_csv(
    "data/weekly_videos_with_transcripts.csv", dtype=dtypes)  # hoặc đã có sẵn
top_week_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   year                       1400 non-null   int64  
 1   week                       1400 non-null   int64  
 2   weekly_score               1400 non-null   float64
 3   weekly_score_rank          1400 non-null   float64
 4   author.uniqueId            1400 non-null   object 
 5   video.id                   1400 non-null   object 
 6   desc                       1399 non-null   object 
 7   video.duration             1400 non-null   float64
 8   hashtags                   1393 non-null   object 
 9   num_hashtags               1400 non-null   int64  
 10  engagement_rate            1400 non-null   float64
 11  video.url                  1400 non-null   object 
 12  transcript                 1332 non-null   object 
 13  takeaway_1                 1297 non-null   objec

## Đọc các file chứa phân tích transcript


In [4]:
# Đường dẫn folder chứa JSON
json_folder = 'data/transcripts'

# Giả sử đây là DataFrame gốc của bạn

# Danh sách lưu kết quả
json_data_list = []

# Lặp qua từng ID trong DataFrame
for video_id in top_week_df['video.id']:
    file_path = os.path.join(json_folder, f"{video_id}.json")
    if not os.path.exists(file_path):
        print(f"⚠️ File không tồn tại: {file_path}")
        continue

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = json.load(f)
        # Thêm video.id vào nội dung JSON để merge được
        content['video.id'] = video_id
        json_data_list.append(content)
    except Exception as e:
        print(f"❌ Lỗi khi đọc file {file_path}: {e}")

# Chuyển thành DataFrame từ danh sách JSON
json_df = pd.DataFrame(json_data_list)

❌ Lỗi khi đọc file data/transcripts/7305699777222626578.json: Expecting value: line 1 column 1 (char 0)
❌ Lỗi khi đọc file data/transcripts/7377334374360026375.json: Expecting value: line 1 column 1 (char 0)
❌ Lỗi khi đọc file data/transcripts/7418147024186445074.json: Expecting value: line 1 column 1 (char 0)


## Tiền xử lý video content trích xuất từ transcript


In [5]:
json_df.columns

Index(['category', 'hook_type', 'structure_style', 'tone_of_voice', 'pacing',
       'has_cta', 'cta_type', 'has_personal_story', 'main_content_focus',
       'speaking_style', 'emotion_keywords', 'video.id'],
      dtype='object')

In [6]:
cleanned_json_df = json_df.copy()

### Xử lý `category`.


In [7]:
json_df['category'].explode().value_counts()

category
Review sản phẩm ăn uống    425
Nấu ăn                     318
Không liên quan ẩm thực    303
Review quán ăn             202
Mukbang                    134
Thử thách ăn uống           37
Mẹo bếp                      3
Kể chuyện                    2
Review địa điểm ăn uống      2
Trải nghiệm cá nhân          1
Ăn thử và phản ứng           1
Thử sản phẩm ăn uống         1
Phản hồi bình luận           1
Name: count, dtype: int64

Cột `category` có label nhiễu do Gemini tự tạo.


In [8]:
def normalize_list_column(value, mapping):
    try:
        parsed = ast.literal_eval(value) if isinstance(
            value, str) and value.startswith('[') else value
        if isinstance(parsed, list):
            return [mapping.get(item, item) for item in parsed]
        else:
            return [mapping.get(parsed, parsed)]
    except:
        return [value]

In [9]:
# Mapping từ bạn đã định nghĩa
category_mapping = {
    'Mẹo bếp': 'Nấu ăn',
    'Kể chuyện': 'Khác',
    'Trải nghiệm cá nhân': 'Khác',
    'Phản hồi bình luận': 'Khác',
    'Review quán ăn': 'Review địa điểm ăn uống',
    'Ăn thử và phản ứng': 'Mukbang',
    'Thử sản phẩm ăn uống': 'Review sản phẩm ăn uống',
    'Không liên quan ẩm thực': 'Audio không liên quan ẩm thực'
}


# Áp dụng chuẩn hoá category
cleanned_json_df['category'] = cleanned_json_df['category'].apply(
    lambda x: normalize_list_column(x, category_mapping))


cleanned_json_df['category'].explode().value_counts()

category
Review sản phẩm ăn uống          426
Nấu ăn                           321
Audio không liên quan ẩm thực    303
Review địa điểm ăn uống          204
Mukbang                          135
Thử thách ăn uống                 37
Khác                               4
Name: count, dtype: int64

### Xử lý cột `hook_type`.


In [10]:
cleanned_json_df['hook_type'].explode().value_counts()

hook_type
Gây tò mò             770
Vào thẳng món         531
Kể chuyện             125
Phản hồi bình luận     94
Đặt câu hỏi            68
Khuyến mãi             15
So sánh                 5
Cảnh báo                1
Name: count, dtype: int64

In [11]:
hook_mapping = {
    'Cảnh báo': 'Gây tò mò',
    # 'So sánh': 'Khác',
    # 'Khuyến mãi': 'Khác'
}

cleanned_json_df['hook_type'] = cleanned_json_df['hook_type'].apply(
    lambda x: normalize_list_column(x, hook_mapping)
)

cleanned_json_df['hook_type'].explode().value_counts()

hook_type
Gây tò mò             771
Vào thẳng món         531
Kể chuyện             125
Phản hồi bình luận     94
Đặt câu hỏi            68
Khuyến mãi             15
So sánh                 5
Name: count, dtype: int64

### Xử lý cột `structure_style`.


In [12]:
cleanned_json_df['structure_style'].explode().value_counts()

structure_style
Cảm nhận từng món           466
Ăn thử và phản ứng          433
Hướng dẫn từng bước         319
Kể chuyện kết hợp review    229
Mô tả kết hợp tương tác     181
Liệt kê món                 111
So sánh món                  19
Name: count, dtype: int64

In [13]:
tone_mapping = {
    'Cảm nhận từng món': 'Ăn thử và phản ứng',
    'Bốc phốt': 'Bất mãn',
    'Ức chế': 'Bất mãn',
    'Bất lực': 'Bất mãn',
    'Bức xúc': 'Bất mãn',
    'Tiếc nuối': 'Bất mãn',
    'Tò mò': 'Khác',
    'Cảm xúc': 'Khác',
    'Tự tin': 'Hào hứng',
    'Nghiêm túc': 'Trung lập',
    'Chia sẻ': 'Khác',
    'Tự nhiên': 'Thân mật',
}

cleanned_json_df['tone_of_voice'] = cleanned_json_df['tone_of_voice'].apply(
    lambda x: normalize_list_column(x, tone_mapping)
)

cleanned_json_df['tone_of_voice'].explode().value_counts()

tone_of_voice
Thân mật      1023
Hào hứng       636
Hài hước       254
Ngạc nhiên      79
Chân thành      24
Bất mãn         23
Trung lập       21
Khác             6
Thất vọng        3
Name: count, dtype: int64

### Xử lý cột `tone_of_voice`.


In [14]:
cleanned_json_df['tone_of_voice'].explode().value_counts()

tone_of_voice
Thân mật      1023
Hào hứng       636
Hài hước       254
Ngạc nhiên      79
Chân thành      24
Bất mãn         23
Trung lập       21
Khác             6
Thất vọng        3
Name: count, dtype: int64

In [15]:
tone_mapping = {
    'Thất vọng': 'Bất mãn',
    'Bốc phốt': 'Bất mãn',
    'Ức chế': 'Bất mãn',
    'Bất lực': 'Bất mãn',
    'Bức xúc': 'Bất mãn',
    'Tiếc nuối': 'Bất mãn',
    'Tò mò': 'Khác',
    'Cảm xúc': 'Khác',
    'Tự tin': 'Hào hứng',
    'Nghiêm túc': 'Trung lập',
    'Chia sẻ': 'Khác',
    'Tự nhiên': 'Thân mật',
}

cleanned_json_df['tone_of_voice'] = cleanned_json_df['tone_of_voice'].apply(
    lambda x: normalize_list_column(x, tone_mapping)
)

cleanned_json_df['tone_of_voice'].explode().value_counts()

tone_of_voice
Thân mật      1023
Hào hứng       636
Hài hước       254
Ngạc nhiên      79
Bất mãn         26
Chân thành      24
Trung lập       21
Khác             6
Name: count, dtype: int64

### Xử lý cột `pacing`.


In [16]:
cleanned_json_df['pacing'].explode().value_counts()

pacing
Thay đổi    660
Nhanh       344
Chậm         95
Name: count, dtype: int64

### Xử lý cột `has_cta`.


In [17]:
cleanned_json_df['has_cta'].explode().value_counts()

has_cta
True     872
False    222
Name: count, dtype: int64

In [18]:
cleanned_json_df['has_cta'] = cleanned_json_df['has_cta'].astype('bool')

### Xử lý cột `cta_type`.


In [19]:
cleanned_json_df['cta_type'].explode().value_counts()

cta_type
Follow           376
Đến ăn thử       354
Comment          236
Xem tiếp         128
Tag bạn bè       121
Lưu công thức    101
Ăn thử            80
Mời ăn thử         6
Tham khảo          3
Liên hệ            2
Like               1
Đặt hàng           1
Bình chọn          1
Bye bye            1
Cmt                1
Gọi điện           1
Khuyến mãi         1
Thử nghiệm         1
Mua hàng           1
Name: count, dtype: int64

In [20]:
cta_mapping = {
    'Đến ăn thử': 'Ăn thử',
    'Mời ăn thử': 'Ăn thử',
    'Tham khảo': 'Lưu công thức',
    'Comment': 'Tương tác',
    'Like': 'Tương tác',
    'Cmt': 'Tương tác',
    'Tag bạn bè': 'Tương tác',
    'Đặt hàng': 'Liên hệ',
    'Bình chọn': 'Khác',
    'Bye bye': 'Khác',
    'Gọi điện': 'Liên hệ',
    'Khuyến mãi': 'Khác',
    'Thử nghiệm': 'Khác',
    'Mua hàng': 'Liên hệ',
}

cleanned_json_df['cta_type'] = cleanned_json_df['cta_type'].apply(
    lambda x: normalize_list_column(x, cta_mapping)
)

cleanned_json_df['cta_type'].explode().value_counts()

cta_type
Ăn thử           440
Follow           376
Tương tác        359
Xem tiếp         128
Lưu công thức    104
Liên hệ            5
Khác               4
Name: count, dtype: int64

### Xử lý cột `has_personal_story`.


In [21]:
cleanned_json_df['has_personal_story'].explode().value_counts()

has_personal_story
True     905
False    189
Name: count, dtype: int64

In [22]:
cleanned_json_df['has_personal_story'] = cleanned_json_df['has_personal_story'].astype(
    'bool')

### Xử lý cột `main_content_focus`.


In [23]:
cleanned_json_df['main_content_focus'].explode().value_counts()

main_content_focus
Món ăn                 797
Cảm nhận khi ăn        708
Sản phẩm ăn uống       405
Trải nghiệm cá nhân    392
Công thức nấu ăn       286
Quán ăn                224
Kinh nghiệm cá nhân      1
Name: count, dtype: int64

In [24]:
cta_mapping = {
    'Kinh nghiệm cá nhân': 'Trải nghiệm cá nhân'
}

cleanned_json_df['main_content_focus'] = cleanned_json_df['main_content_focus'].apply(
    lambda x: normalize_list_column(x, cta_mapping)
)

cleanned_json_df['main_content_focus'].explode().value_counts()

main_content_focus
Món ăn                 797
Cảm nhận khi ăn        708
Sản phẩm ăn uống       405
Trải nghiệm cá nhân    393
Công thức nấu ăn       286
Quán ăn                224
Name: count, dtype: int64

### Xử lý cột `speaking_style`.


In [25]:
cleanned_json_df.columns

Index(['category', 'hook_type', 'structure_style', 'tone_of_voice', 'pacing',
       'has_cta', 'cta_type', 'has_personal_story', 'main_content_focus',
       'speaking_style', 'emotion_keywords', 'video.id'],
      dtype='object')

In [26]:
cleanned_json_df['speaking_style'].explode().value_counts()

speaking_style
Mô tả trực tiếp    907
Độc thoại          591
Kể chuyện          454
Tâm sự              94
Đối thoại giả       77
Name: count, dtype: int64

### Chuyển thành list


In [27]:
import ast


def parse_list_column(value):
    try:
        # Nếu là string dạng list → ép kiểu
        parsed = ast.literal_eval(value) if isinstance(
            value, str) and value.startswith('[') else value
        return parsed if isinstance(parsed, list) else [parsed]
    except:
        return [value]

In [28]:
list_columns = ['category', 'hook_type', 'structure_style', 'tone_of_voice', 'pacing',
                'cta_type', 'main_content_focus', 'speaking_style', 'emotion_keywords']

for col in list_columns:
    cleanned_json_df[col] = cleanned_json_df[col].apply(parse_list_column)

### Merge với `top_week_df`


In [29]:
# Merge vào DataFrame gốc theo 'video.id'
merged_df = top_week_df.merge(cleanned_json_df, on='video.id', how='left')

In [30]:
merged_df.columns

Index(['year', 'week', 'weekly_score', 'weekly_score_rank', 'author.uniqueId',
       'video.id', 'desc', 'video.duration', 'hashtags', 'num_hashtags',
       'engagement_rate', 'video.url', 'transcript', 'takeaway_1',
       'takeaway_2', 'takeaway_3', 'transcript_call_to_action',
       'transcript_curiosity_gap', 'category', 'hook_type', 'structure_style',
       'tone_of_voice', 'pacing', 'has_cta', 'cta_type', 'has_personal_story',
       'main_content_focus', 'speaking_style', 'emotion_keywords'],
      dtype='object')

In [31]:
merged_df = merged_df.drop(columns=['takeaway_1',
                                    'takeaway_2', 'takeaway_3', 'transcript_call_to_action',
                                    'transcript_curiosity_gap'])

## Lấy các cột thống kê


In [32]:
renamed_cols = {
    'author.uniqueId': 'username',
    'video.id': 'video.id',
    "id": "id",
    "video.duration": "duration",
    "statsV2.playCount": "views",  # Lượt xem video
    "statsV2.diggCount": "likes",  # Lượt thích video
    "statsV2.commentCount": "comments",  # Lượt bình luận video
    "statsV2.shareCount": "shares",  # Lượt chia sẻ video
    'statsV2.collectCount': "collects",
    "statsV2.playCount_rank": "views_rank",  # Lượt xem video
    "statsV2.diggCount_rank": "likes_rank",  # Lượt thích video
    "statsV2.commentCount_rank": "comments_rank",  # Lượt bình luận video
    "statsV2.shareCount_rank": "shares_rank",  # Lượt chia sẻ video
    'statsV2.collectCount_rank': "collects_rank",
    'authorStats.diggCount': "user.likes",
    'authorStats.followerCount': "user.followers",
    'authorStats.followingCount': "user.followings",
    'authorStats.heartCount': "user.hearts",
    'authorStats.videoCount': "user.videos",
    'createTime': 'createTime'
}

In [33]:
full_df = pd.read_csv("data/videos_with_weekly_rank.csv", dtype=dtypes)
# full_df = full_df.rename(columns={'id': 'video.id'})
full_df['video.id'] = full_df['id']
full_df = full_df[renamed_cols.keys()]
full_df

,author.uniqueId,video.id,id,video.duration,statsV2.playCount,statsV2.diggCount,statsV2.commentCount,statsV2.shareCount,statsV2.collectCount,statsV2.playCount_rank,statsV2.diggCount_rank,statsV2.commentCount_rank,statsV2.shareCount_rank,statsV2.collectCount_rank,authorStats.diggCount,authorStats.followerCount,authorStats.followingCount,authorStats.heartCount,authorStats.videoCount,createTime
0,ongduymukbang,7467908960897600775,7467908960897600775,28.0,6900000.0,95300.0,2828.0,878.0,5013.0,3,13,3,190,44,3219.0,411000.0,9398.0,6800000.0,774.0,2025-02-05 19:20:55+07:00
1,ongduymukbang,7403358416166571282,7403358416166571282,34.0,9000000.0,162200.0,1350.0,589.0,6752.0,5,14,18,257,37,3219.0,411000.0,9398.0,6800000.0,774.0,2024-08-15 20:31:48+07:00
2,ongduymukbang,7345836549130652946,7345836549130652946,20.0,7400000.0,96800.0,1749.0,577.0,6049.0,8,37,18,214,60,3219.0,411000.0,9398.0,6800000.0,774.0,2024-03-13 20:17:36+07:00
3,ongduymukbang,7483120059301268744,7483120059301268744,45.0,290900.0,4987.0,287.0,56.0,265.0,51,72,16,158,109,3219.0,411000.0,9398.0,6800000.0,774.0,2025-03-18 19:07:44+07:00
4,ongduymukbang,7482376120239787271,7482376120239787271,28.0,465300.0,6130.0,274.0,81.0,442.0,118,221,85,488,326,3219.0,411000.0,9398.0,6800000.0,774.0,2025-03-16 19:00:52+07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70991,dangmukbangbo,7441977141060766996,7441977141060766996,88.0,255100.0,6326.0,67.0,150.0,226.0,323,294,278,493,548,70.0,18200.0,3.0,600900.0,27.0,2024-11-27 22:12:13+07:00
70992,dangmukbangbo,7441452626845142288,7441452626845142288,8.0,69500.0,571.0,16.0,87.0,20.0,628,821,329,551,717,70.0,18200.0,3.0,600900.0,27.0,2024-11-26 12:17:11+07:00
70993,dangmukbangbo,7440860844608458005,7440860844608458005,68.0,115200.0,2477.0,18.0,28.0,113.0,462,569,346,631,675,70.0,18200.0,3.0,600900.0,27.0,2024-11-24 22:00:26+07:00
70994,dangmukbangbo,7440489842275814676,7440489842275814676,71.0,93900.0,2067.0,47.0,193.0,103.0,506,610,317,495,684,70.0,18200.0,3.0,600900.0,27.0,2024-11-23 22:00:47+07:00


Dataframe cuối cùng để phân tích:


In [34]:
# final_df = merged_df.merge(full_df, on=['author.uniqueId', 'video.id', 'video.duration'], how='left').rename(
#     columns=renamed_cols)
final_df = merged_df.merge(
    full_df, on=['author.uniqueId', 'video.id', 'video.duration'], how='left')

In [35]:
final_df.columns

Index(['year', 'week', 'weekly_score', 'weekly_score_rank', 'author.uniqueId',
       'video.id', 'desc', 'video.duration', 'hashtags', 'num_hashtags',
       'engagement_rate', 'video.url', 'transcript', 'category', 'hook_type',
       'structure_style', 'tone_of_voice', 'pacing', 'has_cta', 'cta_type',
       'has_personal_story', 'main_content_focus', 'speaking_style',
       'emotion_keywords', 'id', 'statsV2.playCount', 'statsV2.diggCount',
       'statsV2.commentCount', 'statsV2.shareCount', 'statsV2.collectCount',
       'statsV2.playCount_rank', 'statsV2.diggCount_rank',
       'statsV2.commentCount_rank', 'statsV2.shareCount_rank',
       'statsV2.collectCount_rank', 'authorStats.diggCount',
       'authorStats.followerCount', 'authorStats.followingCount',
       'authorStats.heartCount', 'authorStats.videoCount', 'createTime'],
      dtype='object')

In [36]:
final_df.to_csv("notebooks/preprocess/content_analyse.csv", index=False)